In [12]:
import pandas as pd
import altair as alt
#
#!{sys.executable} -m pip install vega_datasets
from vega_datasets import data
import numpy as np

import sys
!{sys.executable} -m pip install scikit-learn

In [13]:
df = pd.read_csv('billionaires.csv')
df.columns = df.columns.str.replace('.', '_')
df.columns = df.columns.str.replace(' ', '_')
df.head()

<ipython-input-13-63f23e5db2aa>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('.', '_')


,name,rank,year,company_founded,company_name,company_relationship,company_sector,company_type,demographics_age,demographics_gender,...,location_gdp,location_region,wealth_type,wealth_worth_in_billions,wealth_how_category,wealth_how_from_emerging,wealth_how_industry,wealth_how_inherited,wealth_how_was_founder,wealth_how_was_political
0,Bill Gates,1,1996,1975,Microsoft,founder,Software,new,40,male,...,8.100000e+12,North America,founder non-finance,18.5,New Sectors,True,Technology-Computer,not inherited,True,True
1,Bill Gates,1,2001,1975,Microsoft,founder,Software,new,45,male,...,1.060000e+13,North America,founder non-finance,58.7,New Sectors,True,Technology-Computer,not inherited,True,True
2,Bill Gates,1,2014,1975,Microsoft,founder,Software,new,58,male,...,0.000000e+00,North America,founder non-finance,76.0,New Sectors,True,Technology-Computer,not inherited,True,True
3,Warren Buffett,2,1996,1962,Berkshire Hathaway,founder,Finance,new,65,male,...,8.100000e+12,North America,founder non-finance,15.0,Traded Sectors,True,Consumer,not inherited,True,True
4,Warren Buffett,2,2001,1962,Berkshire Hathaway,founder,Finance,new,70,male,...,1.060000e+13,North America,founder non-finance,32.3,Traded Sectors,True,Consumer,not inherited,True,True


In [14]:
df.isnull().sum()

name                         0
rank                         0
year                         0
company_founded              0
company_name                38
company_relationship        46
company_sector              23
company_type                36
demographics_age             0
demographics_gender         34
location_citizenship         0
location_country_code        0
location_gdp                 0
location_region              0
wealth_type                 22
wealth_worth_in_billions     0
wealth_how_category          1
wealth_how_from_emerging     0
wealth_how_industry          1
wealth_how_inherited         0
wealth_how_was_founder       0
wealth_how_was_political     0
dtype: int64

In [15]:
#geo = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')
#geo = geo[['latitude','longitude','country']]
url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
geo = pd.read_csv(url, index_col=0)
geo.reset_index(inplace=True)
geo = geo[['name','alpha-3','country-code']]

df=pd.merge(df, geo, how='left', left_on=['location_country_code'], right_on=['alpha-3'])
df.loc[df['location_citizenship'] == "Taiwan", "country-code"] = geo[geo['alpha-3'] =='TWN']['country-code'].item()
df.loc[df['location_citizenship'] == "Denmark", "country-code"] = geo[geo['alpha-3'] =='DNK']['country-code'].item()
df = df.drop(['name_y','alpha-3'],axis=1)

df.head()

,name_x,rank,year,company_founded,company_name,company_relationship,company_sector,company_type,demographics_age,demographics_gender,...,location_region,wealth_type,wealth_worth_in_billions,wealth_how_category,wealth_how_from_emerging,wealth_how_industry,wealth_how_inherited,wealth_how_was_founder,wealth_how_was_political,country-code
0,Bill Gates,1,1996,1975,Microsoft,founder,Software,new,40,male,...,North America,founder non-finance,18.5,New Sectors,True,Technology-Computer,not inherited,True,True,840.0
1,Bill Gates,1,2001,1975,Microsoft,founder,Software,new,45,male,...,North America,founder non-finance,58.7,New Sectors,True,Technology-Computer,not inherited,True,True,840.0
2,Bill Gates,1,2014,1975,Microsoft,founder,Software,new,58,male,...,North America,founder non-finance,76.0,New Sectors,True,Technology-Computer,not inherited,True,True,840.0
3,Warren Buffett,2,1996,1962,Berkshire Hathaway,founder,Finance,new,65,male,...,North America,founder non-finance,15.0,Traded Sectors,True,Consumer,not inherited,True,True,840.0
4,Warren Buffett,2,2001,1962,Berkshire Hathaway,founder,Finance,new,70,male,...,North America,founder non-finance,32.3,Traded Sectors,True,Consumer,not inherited,True,True,840.0


In [16]:
alt.Chart(df).mark_bar().encode(
    alt.X("sum(wealth_worth_in_billions):Q", scale=alt.Scale(zero=False)),
    alt.Y("location_region"))

alt.Chart(...)

In [17]:

alt.Chart(df[df['year']==2001]).mark_bar().encode(
    alt.X("sum(wealth_worth_in_billions):Q", scale=alt.Scale(zero=False)),
    alt.Y("location_region")
)

alt.Chart(...)

In [18]:
alt.Chart(df[df['year']==2014]).mark_bar().encode(
    alt.X("sum(wealth_worth_in_billions):Q", scale=alt.Scale(zero=False)),
    alt.Y("location_region")
)

alt.Chart(...)

In [19]:
df_ag = df[['wealth_worth_in_billions','location_citizenship','country-code','year']].groupby(
    ['location_citizenship','year']).agg({'wealth_worth_in_billions' : 'sum',
                     'country-code' : 'first'})
df_ag = df_ag.reset_index()
df_ag['log_wealth_worth_in_billions'] = df_ag.wealth_worth_in_billions.apply(lambda x: np.log(x))
df_ag.to_csv('df_ag.csv',index=False)
df_ag

,location_citizenship,year,wealth_worth_in_billions,country-code,log_wealth_worth_in_billions
0,Algeria,2014,3.2,12.0,1.163151
1,Angola,2014,3.7,24.0,1.308333
2,Argentina,1996,6.7,32.0,1.902108
3,Argentina,2001,5.3,32.0,1.667707
4,Argentina,2014,11.3,32.0,2.424803
...,...,...,...,...,...
150,United States,2014,2322.4,840.0,7.750356
151,Venezuela,1996,2.4,862.0,0.875469
152,Venezuela,2001,9.8,862.0,2.282382
153,Venezuela,2014,9.0,862.0,2.197225


In [20]:
world = alt.topo_feature(data.world_110m.url, "countries")

select = alt.binding_select(options=sorted(df_ag.year.unique()), name='Year:')
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=select, init={'year':1996})



background = alt.Chart(world).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    title='Countries over the World',
    width=700,
    height=400
).project('naturalEarth1')




foreground = alt.Chart(df_ag).mark_geoshape().encode(
    color = 'log_wealth_worth_in_billions:Q',
    tooltip = [alt.Tooltip("location_citizenship:N", title="Country"),
               alt.Tooltip("wealth_worth_in_billions:Q", title="wealth")]
).add_selection(select_year
).transform_filter(select_year
).transform_lookup(lookup='country-code',from_=alt.LookupData(world, key='id',fields=["type", "properties", "geometry"])
).project('naturalEarth1'
).properties(
   width=500,
    height=300,
    title='Title'
)




background + foreground

alt.LayerChart(...)

In [32]:
world = alt.topo_feature(data.world_110m.url, "countries")

select = alt.binding_select(options=sorted(df_ag.year.unique()), name='Year:')
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=select)



background = alt.Chart(world).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    title='Countries over the World',
    width=700,
    height=400
).project('naturalEarth1')




foreground = alt.Chart(df_ag).mark_geoshape().encode(
        color=alt.condition('datum.log_wealth_worth_in_billions != null', 
                         'log_wealth_worth_in_billions:Q',
                         alt.value('lightgray'),legend=None
                        ),
        tooltip = [alt.Tooltip("location_citizenship:N", title="Country"),
                alt.Tooltip("wealth_worth_in_billions:Q", title="wealth")]
    ).add_selection(select_year
    ).transform_lookup(lookup='country-code',from_=alt.LookupData(world, key='id',fields=["type", "properties", "geometry"])
    ).project('naturalEarth1'
    ).properties(
    width=500,
        height=300,
        title='Title'
    )

background + foreground


alt.LayerChart(...)

### Draft 1

In [21]:
world = alt.topo_feature(data.world_110m.url, "countries")

slider = alt.binding_range(min=1996, max=2014, step=1, name='Year:')
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=slider)


background = alt.Chart(world).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    title='Countries over the World',
    width=700,
    height=400
).project('naturalEarth1')




foreground = alt.Chart(world).mark_geoshape().encode(
     color=alt.condition('datum.log_wealth_worth_in_billions != null', 
                         'log_wealth_worth_in_billions:Q',
                         alt.value('lightgray'),legend=None
                        ),
        tooltip=[
            alt.Tooltip("location_citizenship:N", title="Country"),
            alt.Tooltip("wealth_worth_in_billions:Q", title="wealth"),
        ],
    ).transform_lookup(
        lookup="id",
        from_=alt.LookupData(df_ag,
                             "country-code", ["log_wealth_worth_in_billions", "wealth_worth_in_billions",
                                                                          "location_citizenship"]),
    ).project(
    type='naturalEarth1'
).properties(
    width=700,
    height=400
)


background + foreground

alt.LayerChart(...)

### Draft 2

In [ ]:
world = alt.topo_feature(data.world_110m.url, "countries")

background = alt.Chart(world).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    title='Countries over the World',
    width=700,
    height=400
).project('naturalEarth1')


foreground = alt.Chart(world).mark_geoshape().encode(
        color=alt.Color("log_wealth_worth_in_billions:Q",
                       legend=alt.Legend(values=['Wealth Worth in Billions'])),
        tooltip=[
            alt.Tooltip("location_citizenship:N", title="Country"),
            alt.Tooltip("wealth_worth_in_billions:Q", title="wealth"),
        ],
    ).transform_lookup(
        lookup="id",
        from_=alt.LookupData(df_ag[df_ag['year']==2001],
                             "country-code", ["log_wealth_worth_in_billions", "wealth_worth_in_billions",
                                                                          "location_citizenship"]),
    ).project(
    type='naturalEarth1'
).properties(
    width=700,
    height=400
)  


background + foreground

foreground = alt.Chart(world).mark_geoshape(stroke="black", strokeWidth=0.15)
    .encode(
        color=alt.Color(
            "log_wealth_worth_in_billions:O", scale=alt.Scale(scheme="orangered"), legend=None,
        ),
        tooltip=[
            alt.Tooltip("location_citizenship:N", title="Country"),
            alt.Tooltip("wealth_worth_in_billions:Q", title="wealth"),
        ],
    )
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(df_ag[df_ag['year']==1996], "country-code", ["log_wealth_worth_in_billions", "wealth_worth_in_billions",
                                                                          "location_citizenship"]),
    ).project(
    type='naturalEarth1'
).properties(
    width=700,
    height=400
)  
)


import altair as alt
from vega_datasets import data
source = alt.topo_feature(data.world_110m.url, "countries")
base_map = (
    alt.Chart(source)
    .mark_geoshape(fill="white", stroke="gray")
    .properties(width=600, height=300)
    .project("naturalEarth1")
)

points = (
    alt.Chart(data1)
    .mark_point()
    .encode(
        latitude="latitude",
        longitude="longitude",
        fill=alt.value("red"),
        size="wealth_worth_in_billions:Q",
        stroke=alt.value(None),
     )
)

final_map = (
    (base_map + points)
    .configure_view(strokeWidth=0)
    .configure_mark(opacity=0.5,)
)
final_map

import altair as alt
from vega_datasets import data
source = alt.topo_feature(data.world_110m.url, 'countries')
alt.Chart(source).mark_geoshape(
    fill='lightgreen',
    stroke='grey',
).encode(
    tooltip='id:N' 
).project(
    'naturalEarth1'
).properties(
    width=600, height=400
).configure_view(stroke=None)

states = alt.topo_feature(data.us_10m.url, 'states')
source = df

foreground = alt.Chart(states).mark_geoshape().encode(
    color=alt.Color('avg_prem:Q')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'id', ['avg'])
).project(
    type='albersUsa'
).properties(
    width=700,
    height=400
)  

background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    title='US State Capitols',
    width=700,
    height=400
).project('albersUsa')

background + foreground

In [ ]:
from vega_datasets import data

source = data.population.url

pink_blue = alt.Scale(domain=('Male', 'Female'),
                      range=["steelblue", "salmon"])

slider = alt.binding_range(min=1900, max=2000, step=10)
#slider = alt.binding_range(1990,1992)
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=slider, init={'year': 2000})

alt.Chart(source).mark_bar().encode(
    x=alt.X('sex:N', title=None),
    y=alt.Y('people:Q', scale=alt.Scale(domain=(0, 12000000))),
    color=alt.Color('sex:N', scale=pink_blue),
    column='age:O'
).properties(
    width=20
).add_selection(
    select_year
).transform_calculate(
    "sex", alt.expr.if_(alt.datum.sex == 1, "Male", "Female")
).transform_filter(
    select_year
).configure_facet(
    spacing=8
)

In [ ]:
alt.Chart(df14).mark_bar().encode(
    x=alt.X('location_region'),
    y=alt.Y('count()')
)

In [ ]:
len()

In [ ]:
df.columns

In [ ]:
df.wealth_how_category.unique()

In [ ]:
df14.location_region.unique()

# new, acquired, subsidiary, merger, franchise, state owned enterprise, privatized, joint venture

In [ ]:
# 2014
Y: wealth_worth_in_billions
X: year - company_founded, (company_type), demographics_age, demographics_gender, location_region, wealth_how_from_emerging, 
    wealth_how_industry, wealth_how_inherited, wealth_how_was_founder, wealth_how_was_political

In [7]:
df14 = df[df['year']==2014]
df14.dropna(inplace=True)
col = ['wealth_worth_in_billions','year','company_founded', 'company_type', 'demographics_age', 'demographics_gender', 
       'location_region', 'wealth_how_from_emerging', 'wealth_how_industry', 'wealth_how_inherited',
       'wealth_how_was_founder', 'wealth_how_was_political']
df14 = df14[col]


d:\Users\tinaf\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
df14.describe()

,wealth_worth_in_billions,year,company_founded,demographics_age
count,1637.000000,1637.0,1637.000000,1637.000000
mean,3.914111,2014.0,1957.122175,60.922419
std,5.769556,0.0,114.861109,17.665722
min,1.000000,2014.0,0.000000,0.000000
25%,1.400000,2014.0,1948.000000,52.000000
50%,2.100000,2014.0,1972.000000,62.000000
75%,3.700000,2014.0,1991.000000,72.000000
max,76.000000,2014.0,2012.000000,98.000000


In [9]:
df14 = df14[(df14['demographics_age']!=0) & (df14['company_founded']!=0) & (df14.wealth_how_industry !='0')]

In [10]:
df14.loc[(df['company_type'] == " new") | (df['company_type'] == "new ") |(df['company_type'] == "new division"),
         'company_type'] = 'new'
df14.loc[(df['company_type'] == " acquired") | (df['company_type'] == "aquired") |(df['company_type'] == "new/aquired"),
         'company_type'] = 'acquired'
df14.loc[df['company_type'] == "new, privitization", 'company_type'] = 'privatization'


In [11]:
df14.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1565 entries, 2 to 2613
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   wealth_worth_in_billions  1565 non-null   float64
 1   year                      1565 non-null   int64  
 2   company_founded           1565 non-null   int64  
 3   company_type              1565 non-null   object 
 4   demographics_age          1565 non-null   int64  
 5   demographics_gender       1565 non-null   object 
 6   location_region           1565 non-null   object 
 7   wealth_how_from_emerging  1565 non-null   bool   
 8   wealth_how_industry       1565 non-null   object 
 9   wealth_how_inherited      1565 non-null   object 
 10  wealth_how_was_founder    1565 non-null   bool   
 11  wealth_how_was_political  1565 non-null   bool   
dtypes: bool(3), float64(1), int64(3), object(5)
memory usage: 126.9+ KB


In [12]:
df14.wealth_how_industry.unique()

array(['Technology-Computer', 'Media', 'Retail, Restaurant', 'Consumer',
       'Diversified financial', 'Real Estate', 'Constrution',
       'Hedge funds', 'Non-consumer industrial', 'Energy',
       'Mining and metals', 'Money Management',
       'Private equity/leveraged buyout', 'Other', 'Technology-Medical',
       'Venture Capital'], dtype=object)

In [13]:
# one-hot encoding
company_type
location_region
wealth_how_inherited
wealth_how_industry
gender

NameError: name 'company_type' is not defined

In [15]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
type_cat_1hot = cat_encoder.fit_transform(df14[['company_type']])
type_cat_1hot.toarray()

cat_encoder.get_feature_names_out()



array(['company_type_acquired', 'company_type_franchise rights',
       'company_type_merger', 'company_type_new',
       'company_type_privatization',
       'company_type_state owned enterprise', 'company_type_subsidiary'],
      dtype=object)

In [ ]:
X = df14[col[1:]]
y = df14[col[0]]

In [ ]:
df14